In [49]:
from dotenv import load_dotenv
import requests
import os
import pandas as pd
load_dotenv()
print("API Key cargada:", os.environ.get('riot_api_key'))



gameName = 'Natanaël%20Cano'
tagLine = 'NATA'
api_key = 'RGAPI-5a6d24a8-95fa-495e-8840-94ec100b9f50'

def get_puuid(gameName=None, tagLine=None, api_key=None):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}'

    response = requests.get(link)

    return response.json()['puuid']

def get_name_and_tag (puuid=None, api_key=None):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'
    
   
    response = requests.get(link)

    gameName = response.json()['gameName']
    tagLine = response.json()['tagLine']
    return f'{gameName}{tagLine}'
    


API Key cargada: RGAPI-45f9032b-bd29-47cf-8815-e2b2f2d18e72


In [ ]:


puuid = get_puuid(gameName=gameName, tagLine=tagLine, api_key=api_key)
print(puuid)

jsPdXjm4uDyQz5yxApVwAEr-NeF8x8QRgzto0VcFyFe0RoJID_ZYQBFXujbry4kwUPXm5UhZXVmn6Q


In [50]:
get_name_and_tag(puuid='56nE7h1a-vwndQ12NUaLVSdnPGJb8gseWdFxlzoVXsKxam2TPmWOSDpormzknIqhbrX64a5BAHD-3A', api_key=api_key)

'Fosuit4444'

In [ ]:
matchtype = 'ranked'
queueid = '420' #Default para SoloQ
start = '0'
count = '10'



def get_matchids (puuid=None, api_key=api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/%27{puuid}%27/ids?queue={queueid}&type={matchtype}&start={start}&count={count}&api_key={api_key}'

    response = requests.get(link)

    return response.json()

In [5]:
history = get_matchids(puuid = puuid, api_key=api_key)
print(history)

['LA1_1637998192', 'LA1_1637347130', 'LA1_1637341373', 'LA1_1637334656', 'LA1_1637328227', 'LA1_1637314825', 'LA1_1636875268', 'LA1_1636864083', 'LA1_1636393076', 'LA1_1636374586']


In [ ]:
from functions_old import get_matchids

puuid = 'jsPdXjm4uDyQz5yxApVwAEr-NeF8x8QRgzto0VcFyFe0RoJID_ZYQBFXujbry4kwUPXm5UhZXVmn6Q'
get_matchids(puuid = puuid, queueid=420, matchtype='ranked', start='0', count='1', api_key='RGAPI-a5407272-0def-46a8-9515-8b2f6033ce47')

['LA1_1642972444']

In [3]:
def winrate(wins, losses):
    """
    Given number of wins and losses, calculate the winrate as a percentage
    """
    return wins / (wins + losses) * 100

In [2]:
def displayWinrates(puuid, history: list, api_key: str):
    wins, losses = (0, 0)
    champion_winrates = {}

    for matchId in history:

        matchUrl = f'https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}'
        response = requests.get(matchUrl)
        matchInfo = response.json()["info"]

        for player in matchInfo["participants"]:
            if player["puuid"] == puuid:
                win = player["win"]
                champion = player["championName"]
                break

        if win:
            wins += 1
            if champion in champion_winrates:
                champion_winrates[champion][0] += 1
            else:
                champion_winrates[champion] = [1, 0, 0]
        else:
            losses += 1
            if champion in champion_winrates:
                champion_winrates[champion][1] += 1
            else:
                champion_winrates[champion] = [0, 1, 0]
        champion_winrates[champion][2] += 1

    # Sort champions in descending order of games
    champion_winrates = dict(
        sorted(champion_winrates.items(), reverse=True, key=lambda x: x[1][2]))
    champion_list = []
    champion_winrates_list = []

    # Overall wins and losses
    print(wins, 'wins', losses, 'losses')

    # Overall winrate to two decimal places
    print('%.2f' % (winrate(wins, losses)) + "%")
    for champion in champion_winrates:
        # Prints champion winrates and checks for plural games
        wins_c, losses_c, games = champion_winrates[champion]
        percentage = '%.2f' % winrate(wins_c, losses_c)
        games = champion_winrates[champion][2]

        champion_list.append(champion)
        champion_winrates_list.append(float(percentage))

        if games == 1:
            print(champion, percentage + '%', games, 'game')
        else:
            print(champion, percentage + '%', games, 'games')

    return champion_list, champion_winrates_list


In [4]:
champion_list, champion_winrates_list = displayWinrates(puuid, history, api_key)
print("Campeones jugados:", champion_list)
print("Winrates respectivos:", champion_winrates_list)

NameError: name 'puuid' is not defined

In [9]:
import time  

def get_participant_id(match_data, puuid):
    for i, participant in enumerate(match_data["metadata"]["participants"]):
        if participant == puuid:
            return str(i + 1)
    return None

for match_id in history:
    try:
        match_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
        match_resp = requests.get(match_url)
        match_data = match_resp.json()

        participant_id = get_participant_id(match_data, puuid)
        if not participant_id:
            print(f"PUUID no encontrado en partida {match_id}")
            continue

        timeline_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}"
        timeline_resp = requests.get(timeline_url)
        timeline_data = timeline_resp.json()

        frames = timeline_data["info"]["frames"]
        if len(frames) > 10:
            total_gold = frames[10]["participantFrames"][participant_id]["totalGold"]
            print(f"[{match_id}] Oro al minuto 10: {total_gold}")
        else:
            print(f"[{match_id}] La partida no llegó al minuto 10.")
        
        time.sleep(1.2)

    except Exception as e:
        print(f"Error en partida {match_id}: {e}")


[LA1_1637998192] Oro al minuto 10: 4790
[LA1_1637347130] Oro al minuto 10: 3851
[LA1_1637341373] Oro al minuto 10: 4342
[LA1_1637334656] Oro al minuto 10: 2708
[LA1_1637328227] Oro al minuto 10: 3321
[LA1_1637314825] Oro al minuto 10: 3114
[LA1_1636875268] Oro al minuto 10: 3465
[LA1_1636864083] Oro al minuto 10: 2922
[LA1_1636393076] Oro al minuto 10: 2994
[LA1_1636374586] Oro al minuto 10: 3118


In [22]:
def get_participant_info(match_data, puuid):
    for i, participant_puuid in enumerate(match_data["metadata"]["participants"]):
        if participant_puuid == puuid:
            participant_id = i + 1
            team_id = match_data["info"]["participants"][i]["teamId"]  # 100 o 200
            return str(participant_id), team_id
    return None, None

def get_team_gold_difference(timeline_data):
    frames = timeline_data["info"]["frames"]
    if len(frames) <= 10:
        return None

    frame_10 = frames[10]["participantFrames"]

    team1_gold = sum([frame_10[str(pid)]["totalGold"] for pid in range(1, 6)])
    team2_gold = sum([frame_10[str(pid)]["totalGold"] for pid in range(6, 11)])

    difference = abs(team1_gold - team2_gold)
    leading_team = 1 if team1_gold > team2_gold else (2 if team2_gold > team1_gold else 0)

    return {
        "team1_gold": team1_gold,
        "team2_gold": team2_gold,
        "difference": difference,
        "leading_team": leading_team  # 1, 2 o 0 (empate)
    }


for match_id in history:
    try:
        match_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
        match_resp = requests.get(match_url)
        match_data = match_resp.json()

        participant_id, team_id = get_participant_info(match_data, puuid)
        if not participant_id:
            print(f"[{match_id}] PUUID no encontrado.")
            continue

        timeline_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}"
        timeline_resp = requests.get(timeline_url)
        timeline_data = timeline_resp.json()

        frames = timeline_data["info"]["frames"]
        if len(frames) <= 10:
            print(f"[{match_id}] La partida no llegó al minuto 10.")
            continue

        # Oro del jugador
        player_gold = frames[10]["participantFrames"][participant_id]["totalGold"]

        # Oro de equipos
        gold_info = get_team_gold_difference(timeline_data)
        if not gold_info:
            print(f"[{match_id}] Error al obtener oro de equipos.")
            continue

        # Determinar si su equipo iba ganando
        puuid_team = 1 if team_id == 100 else 2
        
        time.sleep(1.2) 

    except Exception as e:
        print(f"Error en partida {match_id}: {e}")


In [24]:
import time

def get_participant_info(match_data, puuid):
    for i, participant_puuid in enumerate(match_data["metadata"]["participants"]):
        if participant_puuid == puuid:
            participant_id = i + 1
            team_id = match_data["info"]["participants"][i]["teamId"]
            return participant_id, team_id
    return None, None


In [1]:
def get_gold_at_min10(timeline_data, participant_id):
    frames = timeline_data["info"]["frames"]
    if len(frames) > 10:
        return frames[10]["participantFrames"][str(participant_id)]["totalGold"]
    else:
        return None  # La partida no llegó al minuto 10

In [26]:
def get_team_kills_at_10(timeline_data):
    team_100_kills = 0
    team_200_kills = 0

    for frame in timeline_data["info"]["frames"][:11]:
        for event in frame.get("events", []):
            if event["type"] == "CHAMPION_KILL":
                killer_id = event.get("killerId", 0)
                if killer_id == 0:
                    continue  # torre, súbdito
                team = 100 if killer_id <= 5 else 200
                if team == 100:
                    team_100_kills += 1
                else:
                    team_200_kills += 1

    return team_100_kills, team_200_kills


In [29]:
data = []  # Aquí almacenarás todos los resultados

for match_id in history:
    try:
        match_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
        timeline_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}"

        match_data = requests.get(match_url).json()
        timeline_data = requests.get(timeline_url).json()

        participant_id, team_id = get_participant_info(match_data, puuid)
        if not participant_id:
            print(f"[{match_id}] PUUID no encontrado.")
            continue

        # Determinar si ganó o perdió
        player_info = next((p for p in match_data["info"]["participants"] if p["puuid"] == puuid), None)
        if not player_info:
            print(f"[{match_id}] No se encontró info del jugador.")
            continue
        my_team_win = player_info["win"] 

        gold_10 = get_gold_at_min10(timeline_data, participant_id)
        if gold_10 is None:
            print(f"[{match_id}] No llegó al minuto 10.")
            continue

        # Oro por equipo
        gold_info = get_team_gold_difference(timeline_data)
        if not gold_info:
            print(f"[{match_id}] Error al obtener oro de equipos.")
            continue

        # Asesinatos por equipo
        team_100_kills, team_200_kills = get_team_kills_at_10(timeline_data)
        kill_diff = team_100_kills - team_200_kills
        leading_team_kills = (
            "Azul (100)" if kill_diff > 0 else "Rojo (200)" if kill_diff < 0 else "Empate"
        )

        my_team_label = "Azul (100)" if team_id == 100 else "Rojo (200)"
        my_team_was_ahead_kills = (team_id == 100 and kill_diff > 0) or (team_id == 200 and kill_diff < 0)
        my_team_was_ahead_gold = (team_id == 100 and gold_info["leading_team"] == 1) or (team_id == 200 and gold_info["leading_team"] == 2)

        data.append({
            "match_id": match_id,
            "my_team": my_team_label,
            "gold_min10": gold_10,
            "team1_gold": gold_info["team1_gold"],
            "team2_gold": gold_info["team2_gold"],
            "gold_diff": gold_info["difference"],
            "gold_leading_team": f"Team {gold_info['leading_team']}" if gold_info['leading_team'] else "Empate",
            "my_team_ahead_gold": my_team_was_ahead_gold,
            "team_100_kills": team_100_kills,
            "team_200_kills": team_200_kills,
            "kill_diff": kill_diff,
            "kill_leading_team": leading_team_kills,
            "my_team_ahead_kills": my_team_was_ahead_kills,
            "my_team_win": my_team_win
        })

        time.sleep(1.2)  # Evita el rate limit

    except Exception as e:
        print(f"[{match_id}] Error: {e}")



In [30]:
df = pd.DataFrame(data)
df.to_csv("match_stats_min10.csv", index=False)
df.head()

,match_id,my_team,gold_min10,team1_gold,team2_gold,gold_diff,gold_leading_team,my_team_ahead_gold,team_100_kills,team_200_kills,kill_diff,kill_leading_team,my_team_ahead_kills,my_team_win
0,LA1_1637998192,Rojo (200),4790,15187,17031,1844,Team 2,True,3,5,-2,Rojo (200),True,True
1,LA1_1637347130,Azul (100),3851,17151,16322,829,Team 1,True,8,5,3,Azul (100),True,True
2,LA1_1637341373,Azul (100),4342,16352,16454,102,Team 2,False,6,6,0,Empate,False,True
3,LA1_1637334656,Rojo (200),2708,17868,15054,2814,Team 1,False,8,5,3,Azul (100),False,False
4,LA1_1637328227,Rojo (200),3321,16450,16832,382,Team 2,True,7,7,0,Empate,False,False


In [ ]:
type(data)
team_id
print(data)

[{'match_id': 'LA1_1637998192', 'gold_min10': 4790, 'team_100_kills': 3, 'team_200_kills': 5, 'kill_diff': -2, 'leader_team': 'Rojo (200)', 'my_team': 'Rojo (200)', 'my_team_was_ahead': True}, {'match_id': 'LA1_1637347130', 'gold_min10': 3851, 'team_100_kills': 8, 'team_200_kills': 5, 'kill_diff': 3, 'leader_team': 'Azul (100)', 'my_team': 'Azul (100)', 'my_team_was_ahead': True}, {'match_id': 'LA1_1637341373', 'gold_min10': 4342, 'team_100_kills': 6, 'team_200_kills': 6, 'kill_diff': 0, 'leader_team': 'Empate', 'my_team': 'Azul (100)', 'my_team_was_ahead': False}, {'match_id': 'LA1_1637334656', 'gold_min10': 2708, 'team_100_kills': 8, 'team_200_kills': 5, 'kill_diff': 3, 'leader_team': 'Azul (100)', 'my_team': 'Rojo (200)', 'my_team_was_ahead': False}, {'match_id': 'LA1_1637328227', 'gold_min10': 3321, 'team_100_kills': 7, 'team_200_kills': 7, 'kill_diff': 0, 'leader_team': 'Empate', 'my_team': 'Rojo (200)', 'my_team_was_ahead': False}, {'match_id': 'LA1_1637314825', 'gold_min10': 311

In [28]:
datos_actualizados = agregar_estadisticas_oro_lista(data)
df = pd.DataFrame(datos_actualizados)
print(df.head())


KeyError: 'team_200_gold'

In [3]:
import pygsheets

service_account = pygsheets.authorize(service_account_file='JSONS\\spreadsheet-automator-464102-d215f707cf5b.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1t8TCJxtJJ6CafIUrEC-h_jjYOO-KXs8hfP_4gGWbdCw/edit?usp=sharing')

In [8]:
test = sheet.worksheet('title', 'Test')

<Worksheet 'Test' index:0>

In [11]:
test_df = test.get_as_df()

In [13]:
test_df['random_numbers'] = 1

In [16]:
test.set_dataframe(test_df, 'A1', copyhead=True)

In [1]:
division = 'I'
tier = 'DIAMOND'
page = '1'
ladder_csv = 'ladder_apex_csv.csv'
api_key = 'RGAPI-0de3a4cf-e0c0-4920-9b73-b54b1dbdfe5b'

In [54]:
import requests
def get_population(tier, division, page, api_key):
    link = f'https://la1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}&api_key={api_key}'
    response = requests.get(link)
    leaguev4 = response.json()

    ladder = [summ['puuid'] for summ in leaguev4]

    return ladder



In [5]:
from functions import get_gm_ladder, get_chall_ladder

gm_ladder = get_gm_ladder(api_key=api_key)
ch_ladder = get_chall_ladder(api_key=api_key)


In [6]:
apex_ladder = ch_ladder + gm_ladder
print(len(apex_ladder))

450


In [7]:
import pandas as pd
df_ladder = pd.DataFrame(data = apex_ladder, columns=['puuid_apex'])

El siguiente bloque toma el ladder [D1 Actualizado 21/08/2025 11:39 : Apex actializado 22/08/2025 12:36] y lo envía a un csv, dado que es susceptible a cambiar inesperadamente.

In [8]:
import pandas as pd

df_ladder.to_csv(ladder_csv, index=False)
df_ladder.tail()

,puuid_apex
445,gmMDlVphtL9cNonwpPm1lIhDH8UYNR10JxItMXJWRtlMYY...
446,x193A-0bzR_3YXtwfL0uI_vVLT_G-0QqSBL0eHsvLUmhRc...
447,FURAZHyYNtReaMoOXajFU81iq9Zud-fo9Fx9Gdi_GcSec7...
448,lzCzizb-eTBUfw5KnmESxyilcxkvgrhX-bPkT1nFC5yaij...
449,1addd1iZj80ckGhe8OdkgknQrLi3C9jq5f79lML5qIwo53...


In [1]:
import pandas as pd
df_ladder = pd.read_csv('ladder_apex_csv.csv')
df_ladder.sample(n=5)

,puuid_apex
221,DveQ8Lr_8uWvxk_lmH8wEZEBSaXN6rsq4Z7U0QG32uNGGn...
132,4OSpbn2oprwq1oRgW5hfwAA42of9YPKuBt2t8h4d8lZjZ0...
31,-IVvSpJSXTBLO6SaV7ggL4d4gDb2NsEObJv3RH6n00Mr7m...
243,gEfJWiP6zgD1f-yfcvVybrKkrGuPT_a7nX6VJxx98kxhl8...
81,q2BCuCdMLBaoM_4oErRwEKwFwxOt8JXdg0fEA866PVP4tF...


In [2]:
df_ladder.tail()

,puuid_apex
445,gmMDlVphtL9cNonwpPm1lIhDH8UYNR10JxItMXJWRtlMYY...
446,x193A-0bzR_3YXtwfL0uI_vVLT_G-0QqSBL0eHsvLUmhRc...
447,FURAZHyYNtReaMoOXajFU81iq9Zud-fo9Fx9Gdi_GcSec7...
448,lzCzizb-eTBUfw5KnmESxyilcxkvgrhX-bPkT1nFC5yaij...
449,1addd1iZj80ckGhe8OdkgknQrLi3C9jq5f79lML5qIwo53...


In [5]:
ladder_list = list(df_ladder['puuid_apex'])
ladder_list

['piFRZ81GKwj0KeNNDMbq2dnOKD7jH0mLYlDSYFO-NIWiyGA-Ex5m_jrGCI7sBJH8EKEHM-d8QfLGMw',
 '9xCjku0YSxjIxoZWO3cLdcek-UEaRCciCQr5ohcsvltVIEZugbp_S-jhAk2zri-5MkUOyKDaBYySpA',
 'GteNvM5_r9hGEDGEhGIh8GlF0GJNsgVnDwSUfhAsqeVMKO7KcNeI2NyK2WI-eN6b6dbU_u0c4ByXOw',
 '7snahNIbSg8UzrdMm2h6l2Oj6jL7Xds9wfgOAtv0ICM2_p0XvxclWYJMLv_DuPkp4uPCoo9XXDYd6w',
 'PVGhHaCBXSayPinODOAjWm1ASrv3WaPftujdJ2rpUOs2ZZcXhAMygGgC6qcBlDkUpA8NqXON_Mm2Cw',
 'UDfLPeCieRpHal1dd396nbXDaU7JWn2KZJ424Bn70szoZ8JsJi2p-DJkPKwCQw1t1ny4SvDN2Cigwg',
 'b0NR7wKnP-QU9wMgRE5SDFB9EEk9fcDcAgFIqSMsWZ5hBoloPxA38JnAOOc5oelHVkSu_7phFjwp7Q',
 'LbjpIlBO5L4v5Km4Dbil24ZCIhmRYSv3J9jslYqQ5cPjp7xy_g7M7M4h-fwfJLKbI7d3_1YSg0EEaw',
 'DbXiULoMRCRRGxg-iUnn5ucS09uevuVKQREdTtxb8blKwa7l14yikIFC55hvYHGI6jdmAIMfGJvfew',
 '03Xjohys6MjqYaNGbd3xlyPhx2RX0wQ33KnqwaWNawUci9SLjHUUYg6C6xDpdpB5BwemnHLBNN-jhw',
 'WRbZjA2rag1RY5byLIctOOSoEwyudPa_nyHtxIdkXmtQT2JGKl6mFpE1FSvo0OPS-jJ-LnSZq51MCQ',
 'l8eIdNBSW5NqMc52h9FAuRdQbqGw-OaDf4l9lxW8e5KhMJQT5IsCivDgABasU42K7v6UuJbs6ADSdQ',
 '0U

In [1]:
matchtype = 'ranked'
queueid = '420' #Default para SoloQ
start = '15'
count = '45'
api_key = 'RGAPI-7b7cfd46-736c-4ff4-8b32-bde002ca540e'
csv_path = 'sample_csv.csv'

El siguiente bloque pasa un puuid a la vez y recopila 11 partidas de cada jugador. Esto nos dará la muestra para el modelo.
NOTA: Para una muestra de 204 PUUID y un time.sleep(1.2) tuvo una duración total de 27.54mins, proceder con precaución y tiempo de sobra.

In [ ]:
from functions import get_matchids, get_df_data
import time

all_data = []
seen_matches = set()
all_match_ids = []

for summ in ladder_list:
    matches = list(set(get_matchids(
        puuid=summ,
        queueid=queueid,
        matchtype=matchtype,
        start=start,
        count=count,
        api_key=api_key
    )))
    
    new_matches = [m for m in matches if m not in seen_matches]
    if not new_matches:
        continue

    # ⚠️ usamos una copia "temporal" del CSV que no se mezcla aún
    data_sample = get_df_data(history=new_matches, puuid=summ, api_key=api_key, csv_path="temp.csv")
    
    # añadimos SOLO las filas de los matches nuevos
    all_data.extend(data_sample)
    seen_matches.update(new_matches)
    
    time.sleep(1.7)

# --- al final, construir DataFrame único ---
df_sample = pd.DataFrame(all_data)

# eliminar duplicados por match_id
df_sample.drop_duplicates(subset=["match_id"], inplace=True)

# guardar en el CSV definitivo
df_sample.to_csv("sample_apex_csv.csv", index=False)


[LA1_1634745240] Error al obtener oro de equipos.
[LA1_1636003434] Error al obtener oro de equipos.
[LA1_1639552914] Error al obtener oro de equipos.
[LA1_1634809108] Error al obtener oro de equipos.
[LA1_1640918813] Error: 'metadata'
[LA1_1638371732] Error: 'metadata'
[LA1_1638100150] Error: 'metadata'
[LA1_1636929106] Error: 'metadata'
[LA1_1638216532] Error: 'metadata'
[LA1_1640942628] Error: 'metadata'
[LA1_1638556621] Error: 'metadata'
[LA1_1640912372] Error: 'metadata'
[LA1_1642771475] Error: 'metadata'
[LA1_1642584534] Error: 'metadata'
[LA1_1638181900] Error: 'metadata'
[LA1_1638680230] Error: 'metadata'
[LA1_1642822161] Error: 'metadata'
[LA1_1637853999] Error: 'metadata'
[LA1_1642502781] Error: 'metadata'
[LA1_1642555823] Error: 'metadata'
[LA1_1642832843] Error: 'metadata'
[LA1_1642594833] Error: 'metadata'
[LA1_1638248545] Error: 'metadata'
[LA1_1638095175] Error: 'metadata'
[LA1_1637634825] Error: 'metadata'
[LA1_1642797698] Error: 'metadata'
[LA1_1642511698] Error: 'metad

In [27]:
df_sample = pd.DataFrame(all_data)
df_sample = df_sample.drop_duplicates(subset=["match_id"], keep="first")
df_sample.to_csv("sample_csv.csv", index=False)

In [2]:
import pandas as pd

df = pd.read_csv('sample_apex_csv.csv')
api_key = 'RGAPI-7b7cfd46-736c-4ff4-8b32-bde002ca540e'

In [3]:
import requests
import pandas as pd
import time
from functions import get_tower_fb, get_drake_fb, get_herald_fb

def enrich_with_objectives(df, api_key):
    """
    Enriquecer un DataFrame con primeras torres, dragones y heraldos.
    Agrega 3 columnas: tower_fb, drake_fb, herald_fb
    """
    objectives_data = []

    for match_id in df["match_id"]:
        try:
            # Obtener timeline
            timeline_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}"
            timeline_data = requests.get(timeline_url).json()

            tower_fb = get_tower_fb(timeline_data)
            drake_fb = get_drake_fb(timeline_data)
            herald_fb = get_herald_fb(timeline_data)

            objectives_data.append({
                "match_id": match_id,
                "tower_fb": tower_fb,
                "drake_fb": drake_fb,
                "herald_fb": herald_fb
            })

            time.sleep(1.2)  # evitar rate limit

        except Exception as e:
            print(f"[{match_id}] Error en enrich_with_objectives: {e}")
            objectives_data.append({
                "match_id": match_id,
                "tower_fb": None,
                "drake_fb": None,
                "herald_fb": None
            })

    # Convertir en DataFrame
    df_obj = pd.DataFrame(objectives_data)

    # Merge con el original
    df_merged = df.merge(df_obj, on="match_id", how="left")

    # Reordenar columnas: insertar después de 'my_team_ahead_kills' y antes de 'my_team_win'
    cols = list(df_merged.columns)
    insert_at = cols.index("my_team_win")  # antes de my_team_win
    new_cols_order = (
        cols[:insert_at] +
        ["tower_fb", "drake_fb", "herald_fb"] +
        cols[insert_at:]
    )

    df_final = df_merged[new_cols_order]

    return df_final


In [6]:
df_enriched = enrich_with_objectives(df = df, api_key=api_key)

In [7]:
df_enriched.to_csv('sample_apex_enriched_csv.csv', index = False)